<a href="https://colab.research.google.com/github/young-lu/MovieSentiment/blob/main/RL_for_Movie_Review_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF RL Model

*    Policy Gradient with LSTM/BERT model
*    comparison to normal BERT fine tuning


### basic classification fine-tuning



In [ ]:
# from nltk import sent_tokenize as sentence_tokenize
# import transformers
# from transformers import BertTokenizer
# from keras.preprocessing.sequence import pad_sequences

In [1]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is NOT AVAILABLE


In [2]:
train_data, test_data = tfds.load('imdb_reviews', split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

"""

dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

"""

KeyboardInterrupt: ignored

In [38]:
model = tf.keras.Sequential()
'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1", output_shape=[20], input_shape=[], dtype=tf.string, trainable=True)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam',loss=tf.losses.BinaryCrossentropy(from_logits=True),metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [39]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [40]:
type(x_val)

numpy.ndarray

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
results = model.evaluate(test_data, test_labels)

print(results)

Epoch 1/40
30/30 [==============================] - 2s 58ms/step - loss: 0.7434 - accuracy: 0.5263 - val_loss: 0.6633 - val_accuracy: 0.5984
Epoch 2/40
30/30 [==============================] - 2s 53ms/step - loss: 0.6431 - accuracy: 0.6335 - val_loss: 0.6200 - val_accuracy: 0.6558
Epoch 3/40
30/30 [==============================] - 2s 53ms/step - loss: 0.5964 - accuracy: 0.6854 - val_loss: 0.5812 - val_accuracy: 0.7020
Epoch 4/40
30/30 [==============================] - 2s 53ms/step - loss: 0.5518 - accuracy: 0.7329 - val_loss: 0.5427 - val_accuracy: 0.7397
Epoch 5/40
30/30 [==============================] - 2s 52ms/step - loss: 0.5066 - accuracy: 0.7675 - val_loss: 0.5054 - val_accuracy: 0.7648
Epoch 6/40
30/30 [==============================] - 2s 52ms/step - loss: 0.4641 - accuracy: 0.8007 - val_loss: 0.4697 - val_accuracy: 0.7917
Epoch 7/40
30/30 [==============================] - 2s 52ms/step - loss: 0.4224 - accuracy: 0.8262 - val_loss: 0.4388 - val_accuracy: 0.8044
Epoch 8/40
30

## BERT model 

### imports, get_datasets()

In [1]:
!pip3 install --quiet tensorflow
!pip3 install --quiet tensorflow_text
!pip install -q tf-models-official


In [2]:
'''
imports
'''

import os, sys, re, shutil
import numpy as np
import tensorflow_probability as tfp
import tensorflow_datasets as tfds
from tensorflow.python.ops import string_ops
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer
import matplotlib.pyplot as plt

tfds.disable_progress_bar()

tf.get_logger().setLevel('ERROR')
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print('GPU NAME:\n\t',tf.config.list_physical_devices('GPU'))
'''
train_data, test_data = tfds.load('imdb_reviews', split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)
'''
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Version:  2.3.0
Eager mode:  True
Hub version:  0.10.0
GPU is NOT AVAILABLE
GPU NAME:
	 []


In [5]:
# create the train/validation split
def get_datasets(batch=32, seed=42):
  AUTOTUNE = tf.data.experimental.AUTOTUNE
  batch_size = batch
  seed = seed

  raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
      'aclImdb/train',
      batch_size=batch_size,
      validation_split=0.2,
      subset='training',
      seed=seed)

  class_names = raw_train_ds.class_names
  train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)


  val_ds = tf.keras.preprocessing.text_dataset_from_directory(
      'aclImdb/train',
      batch_size=batch_size,
      validation_split=0.2,
      subset='validation',
      seed=seed)

  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

  test_ds = tf.keras.preprocessing.text_dataset_from_directory(
      'aclImdb/test',
      batch_size=batch_size)

  test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

  return train_ds, val_ds, test_ds

# # example
#
# BS=32
# train, validation, test = get_datasets(batch=BS)
#

In [10]:
encoder_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
preprocessor_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1'

### BERT model functions

In [6]:
# tutorial build the model
def build_bert_model():
  encoder_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
  preprocessor_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1'

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(preprocessor_url, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  # encoder same as bert_model
  encoder = hub.KerasLayer(encoder_url, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [7]:
def compile_bert_model(model,
                       training_set,
                       loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                       metric=tf.metrics.BinaryAccuracy(),
                       lr=3e-5,
                       optim='adamw'):
  loss = loss
  # loss = tf.keras.losses.mean_squared_error(from_logits=True)
  metrics = metric
  init_lr = lr
  epochs = 1


  steps_per_epoch = tf.data.experimental.cardinality(training_set).numpy()
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(0.1*num_train_steps)


  # optimizer = optimization.create_optimizer(
  #     init_lr=init_lr,
  #     num_train_steps=num_train_steps,
  #     num_warmup_steps=num_warmup_steps,
  #     optimizer_type='adamw'
  # )
  optim = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=int(num_train_steps*.13),
      optimizer_type='adamw'
  )
  model.compile(optimizer=optim,loss=loss,metrics=metrics)

'''
# example

bertmodel = build_bert_model()
compile_bert_model(model=bertmodel,training_set=train_ds)

'''

'\n# example\n\nbertmodel = build_bert_model()\ncompile_bert_model(model=bertmodel,training_set=train_ds)\n\n'

In [49]:
BATCH_SIZE=64
train_ds, val_ds, test_ds = get_datasets(batch=BATCH_SIZE)
cfm = build_bert_model()
compile_bert_model(model=cfm,
                   training_set=train_ds)

text_test = tf.constant(['This movie was great, and I think that everyone should watch it! I am a massive fan of time travel comedies.',
             'this was a very bad movie. I have never been a fan of Sherlock Holmes or Watson, but I thought I would give this kooky caper a try. What a mistake!!!'])
labels_test = tf.constant([1.0, 0.0])


Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


Exception ignored in: <bound method CapturableResourceDeleter.__del__ of <tensorflow.python.training.tracking.tracking.CapturableResourceDeleter object at 0x7f2fe332f160>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/training/tracking/tracking.py", line 202, in __del__
    self._destroy_resource()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 823, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/def_function.py", line 697, in _initialize
    *args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/eager/function.py", line 2855, in _get_concrete_function_internal_garbage_collected
    graph_function, _, _ = self._maybe

Train with  ```model.fit()``` on full Training Set



In [51]:
with tf.device('/device:GPU:0'):
  history = cfm.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=1)

print(history.history)

313/313 [==============================] - 136s 433ms/step - loss: 0.4804 - binary_accuracy: 0.7487 - val_loss: 0.3862 - val_binary_accuracy: 0.8144
{'loss': [0.48041561245918274], 'binary_accuracy': [0.7487499713897705], 'val_loss': [0.3862094581127167], 'val_binary_accuracy': [0.8144000172615051]}


Evaluate on Test Set

In [52]:
test_loss, test_acc = cfm.evaluate(test_ds)
print("test loss = {}, test accuracy = {}".format(test_loss,test_acc))

391/391 [==============================] - 67s 171ms/step - loss: 0.3821 - binary_accuracy: 0.8182
test loss = 0.3820759057998657, test accuracy = 0.8182399868965149


In [53]:
history.history.keys()
# print((history.history['binary_accuracy']))

dict_keys(['loss', 'binary_accuracy', 'val_loss', 'val_binary_accuracy'])

In [ ]:
test_loss, test_accuracy = classifier_model.evaluate(test_ds)
print(f'Loss: {test_loss}')
print(f'Accuracy: {test_accuracy}')


782/782 [==============================] - 70s 89ms/step - loss: 0.3607 - binary_accuracy: 0.8317
Loss: 0.36073803901672363
Accuracy: 0.8316799998283386


## LSTM
* Training LSTM with RL architecture

### step and train functions

In [4]:
DEFAULT_STRIP_REGEX = r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']'
'''
def step(model, X):
  # this function uses the LSTM to delete words from sentences
  # 
  # for each word, the LSTM produces a policy as a bicategoriical distribution object
  # use each word's policy to sample an action (0==keep, 1==delete)
  # collect the summed log probabilities for each action to use as update weights
  # 
  # return the outputs for each word in each sentence in the batch X
  # return the actions for each sentence in the batch X
  # return the summed log probabilities for each sentence in the batch X
  batch_logits = model(X)
  batch_actions =[[] for _ in batch_logits]
  log_prob_sums =[None for _ in batch_logits]
  new_X = []

  for idx,sentence in enumerate(batch_logits):
    temp_log_probs = []
    start=time.time()
    for probs in sentence:
      # make a distribution for each word
      pol = tfp.distributions.Categorical(probs=tf.nn.softmax(probs))
      # sample from the distribution
      act = pol.sample()
      # save the action (0=keep,1=delete)
      batch_actions[idx].append(act)
      # with that action, get the log_prob
      log_prob = pol.log_prob(act)
      # save that log_prob to a list 
      temp_log_probs.append(log_prob)
    # if idx==0: print("\tpolicy, action sampling, and log_prob time : ",time.time()-start)
    # sum the log probs for this sentence and add to list
    log_prob_sums[idx] = sum(temp_log_probs)

    x = str(X[idx].numpy(),'utf-8')
    acts = batch_actions[idx]
    
    tmp_str = string_ops.regex_replace(x,DEFAULT_STRIP_REGEX,'')
    tmp_str = str(tmp_str.numpy(), 'utf-8').strip()
    tmp_list = [i for i in tmp_str.split(' ') if i]

    if (len(tmp_list) > len(acts)):
      print("ERROR: REGEX INSUFFICIENT")
      sys.exit()
      # _templist = [i for i in tmp_str.split(' ') if i]
      # if len(acts) != len(_templist):
      #   print("ERROR: REGEX INSUFFICIENT")
      #   print('_templist len : ',len(_templist))
      #   print(_templist)
      #   print('acts len : ', len(acts))
      #   sys.exit()
      # # print('{} ERROR: more tokens than actions...'.format(idx))
    new_list = []
    for i,toke in enumerate(tmp_list):
      if acts[i] != 1:
        new_list.append(toke)
    new_X.append(' '.join(new_list))
    if idx==0: print("\tsentence action and word-removal time : ",time.time()-start)

  return batch_logits, new_X, log_prob_sums
'''


def step2(model, X):
  batch_logits = model(X)
  batch_actions =[[] for _ in batch_logits]
  log_prob_sums =[0.0 for _ in batch_logits]
  _log_prob_sums =[None for _ in batch_logits]
  new_X = []
  words_removed = []
  for idx,sentence in enumerate(batch_logits):
    temp_log_probs = []
    start=time.time()

    pol = tfp.distributions.Bernoulli(logits=sentence)
    act = pol.sample()
    batch_actions[idx] = [ac for ac in act]
    log_probs = [pol.log_prob(ac) for ac in act]
    lpkeep = pol.log_prob([0])
    lpdel = pol.log_prob([1])

    onect=0
    zeroct=0
    lp_list = []
    for i,ac in enumerate(act):
      if int(ac) == 1:
        lp_list.append(lpdel[i])
        onect+=1
      else:
        lp_list.append(lpkeep[i])
    #     zeroct+=1
    # print(onect,"/",zeroct)
    log_prob_sums[idx] = sum(lp_list)
    x = str(X[idx].numpy(),'utf-8')
    acts = batch_actions[idx]
    tmp_str = string_ops.regex_replace(x,DEFAULT_STRIP_REGEX,'')
    tmp_str = str(tmp_str.numpy(), 'utf-8').strip()
    tmp_list = [i for i in tmp_str.split(' ') if i]
    if (len(tmp_list) > len(acts)):
      print("ERROR: REGEX INSUFFICIENT")
      sys.exit()
    new_list = []
    addct=0
    rem=0
    for i,toke in enumerate(tmp_list):
      if acts[i] != 1:
        new_list.append(toke)
      else:
        rem+=1
    words_removed.append(rem)
    new_X.append(' '.join(new_list))
    # if idx==0: 
      # print("\tsentence action and word-removal time : ",time.time()-start)
      # print(X[0])
      # print(new_X[0])

  return batch_logits, new_X, log_prob_sums, np.mean(words_removed)


def remove_words(lstm, X):
  batch_logits = lstm(X)
  batch_actions =[[] for _ in batch_logits]
  new_X = []
  stats=[]
  for idx,sentence in enumerate(batch_logits):
    start=time.time()
    temp_stats = [0,len(sentence)]
    pol = tfp.distributions.Bernoulli(logits=sentence)
    act = pol.sample()
    batch_actions[idx] = [ac for ac in act]

    x = str(X[idx].numpy(),'utf-8')
    acts = batch_actions[idx]
    tmp_str = string_ops.regex_replace(x,DEFAULT_STRIP_REGEX,'')
    tmp_str = str(tmp_str.numpy(), 'utf-8').strip()
    tmp_list = [i for i in tmp_str.split(' ') if i]
    if (len(tmp_list) > len(acts)):
      print("ERROR: REGEX INSUFFICIENT")
      sys.exit()
    new_list = []
    addct=0
    for i,toke in enumerate(tmp_list):
      if acts[i] != 1:
        new_list.append(toke)
      else:
        temp_stats[0] +=1
    stats.append(temp_stats)
    new_X.append(' '.join(new_list))
  return new_X, stats




In [21]:
import sys
import time
# 
# policy gradient (REINFORCE) training loop with LSTM model and BERT sentiment classifier
# classifier must be compiled!
# 
def train_vpg(lstm, optimizer, classifier, training_set, validation_set, epochs=5):
  total_start=time.time()
  stepct=0
  epoch_avg_words_del=[]
  # with tf.device('/device:GPU:0'):
  for e in range(epochs):
    epoch_losses=[]
    counter=0
    trainlen = len(training_set)
    avg_words_removed = []
    total_batches=len(training_set)
    for idx,ex_lab in enumerate(training_set):
    # for idx,ex_lab in enumerate(training_set.take(100)):
      starttime=time.time()
      example, label = ex_lab[0], ex_lab[1]
      counter+=1
      print('epoch={}\tbatch={}/{}'.format(e,counter,total_batches))
      stepct+=1
      # run through the lstm
      logits, new_example, logprob_sums = None, None, None
      with tf.GradientTape() as t:
        start = time.time()
        # logits, new_example, logprob_sums = step(lstm, example)
        logits, new_example, logprob_sums, deleted = step2(lstm, example)
        avg_words_removed.append(deleted)
        if (idx%25==0):
          print("batch={} words deleted : {}".format(idx,deleted))
        # logging for curiosity
        if e==0 and idx==0:
          print('step time for one batch: {:.4f}'.format(time.time()-start))

        # get BERT evaluations on these
        start=time.time()
        new_outs = classifier(tf.constant(new_example))
        # get bert evaluation on full sentences
        baseline_outs = classifier(example)

        if e==0 and idx==0:
          print('BERT processing time for one batch : {:.4f}'.format(time.time()-start))

        # reward LSTM with the improvement over the original sentences
        baseline_loss = tf.keras.losses.mean_squared_error(baseline_outs,tf.cast(label,float))
        loss_object = tf.keras.losses.mean_squared_error(new_outs,tf.cast(label,float))
        # loss_object = tf.keras.losses.binary_crossentropy(new_outs,tf.cast(label,float))
        adjusted_loss = loss_object-baseline_loss
        # print("loss_x' - loss_base : ",np.array(adjusted_loss))
        epoch_losses.append(adjusted_loss.numpy())

        # use BERT's loss for update:
        # use the negative losses to weight the log_probs
        weighted_logprobs = -(adjusted_loss)*logprob_sums
        # end with tf.GradientTape()

      # train bert by fitting on new_examples
      # this way, BERT and the LSTM train together
      with tf.device('/device:GPU:0'):
        history = classifier.fit(x=tf.constant(new_example),
                                  y=label,
                                  # validation_split=.2,
                                  batch_size=len(new_example),
                                  epochs=2,
                                  verbose=(idx%25==0))
      # print("BERT training accuracy : ", np.mean(history.history['binary_accuracy']))

      # get gradients and apply update
      gradients = t.gradient(weighted_logprobs, lstm.trainable_variables)
      optimizer.apply_gradients(zip(gradients,lstm.trainable_variables))

      epoch_avg_words_del.append(np.mean(avg_words_removed))

      if idx == 1: print('batch time {:.2f}'.format(time.time()-starttime))
      # end for
    print('\n_____________________________')
    print('[{}]\t avg training loss : {}'.format(e,np.mean(epoch_losses)))
    # print('[{}]\t total train inloss : {}'.format(e,np.sum(epoch_losses)))
    with tf.device('/device:GPU:0'):
      val_loss, val_acc = classifier.evaluate(validation_set)
    print("[{}] BERT validation loss     :\t{}\n\t\tvalidation accuracy :\t{}".format(idx,val_loss, val_acc))
    print('average words removed: ',epoch_avg_words_del)

    '''
    # validate 
    validate()
    '''

    # end for
  print('DONE. {:.4f}'.format(time.time()-total_start))

In [10]:

def evaluate_lstm_model(lstm, classifier, testing_set):
  test_x=[]
  test_y=[]
  total=len(testing_set)
  stats = []
  for idx,example in enumerate(testing_set):
    test_y.append(example[1].numpy())
    new_x, _stats = remove_words(lstm_model,example[0])
    stats.append(_stats) 
    test_x.append(np.array(new_x))
    print('LSTM processing batch',idx,'/',total-1)
    if idx==0:
      print("Sample LSTM sentence output")
      print(example[0][:5])
      print(new_x[:5])
  test_loss, test_acc = classifier.evaluate(x=test_x,y=test_y)
  return test_loss, test_acc, stats


### setup

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BUFFER_SIZE = 1000 # vocabulary size?
BATCH_SIZE = 32
seed = 22

train_ds, val_ds, test_ds = get_datasets(batch=BATCH_SIZE)
# lstm_train_ds, lstm_val_ds, lstm_train_ds = get_datasets()

text_test = tf.constant(['This movie was great, and I think that everyone should watch it! I am a massive fan of time travel comedies.',
             'this was a very bad movie. I have never been a fan of Sherlock Holmes or Watson, but I thought I would give this kooky caper a try. What a mistake!!!'])
labels_test = tf.constant([1.0, 0.0])

VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_ds.map(lambda text, label: text))


NameError: ignored

In [ ]:
# instantiate the BERT model and classify on two sentences
classifier_model = build_bert_model()
compile_bert_model(model=classifier_model,
                   training_set=train_ds)
# with tf.device('/device:GPU:0'):
#   history = classifier_model.fit(x=text_test, y=labels_test, batch_size=1, epochs=2)


In [28]:
# instantiate LSTM 
lstm_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1,activation=None)
])

lstm_optim = tf.optimizers.Adam()

### train_lstm_vpg 

train LSTM on dataset 
* tune BERT, then train LSTM

In [14]:
with tf.device('/device:GPU:0'):
  history = classifier_model.fit(x=train_ds,
                                validation_data=val_ds.shard(10,0),
                                epochs=1)
  print(history.history)


625/625 [==============================] - 127s 203ms/step - loss: 0.4441 - binary_accuracy: 0.7764 - val_loss: 0.3782 - val_binary_accuracy: 0.8223
{'loss': [0.4441312849521637], 'binary_accuracy': [0.7763500213623047], 'val_loss': [0.37818530201911926], 'val_binary_accuracy': [0.822265625]}


example of accuracy post-tuning

In [29]:
with tf.device('/device:GPU:0'):
  loss, acc = classifier_model.evaluate(test_ds.shard(30,0))
print(loss, acc)

27/27 [==============================] - 2s 79ms/step - loss: 0.3794 - binary_accuracy: 0.8113
0.3794430196285248 0.8113425970077515


train LSTM with VPG
* reward to LSTM = `-(loss_short - loss_original)`
* weighted by `sum(action_log_probs)`

In [30]:
train_vpg(lstm_model, lstm_optim, classifier_model, train_ds, val_ds.shard(10,2), epochs=1)
print('\n_______________________________________________________')
test_loss, test_accuracy, stats = evaluate_lstm_model(lstm_model, classifier_model, test_ds.shard(690,1))
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
means=[]
for batch in stats:
  batch_means=[]
  for pair in batch:
    batch_means.append(pair[0])
  means.append(np.mean(batch_means))
print("mean deletions: ",np.mean(means))
print('\n_______________________________________________________')
test_loss, test_accuracy = classifier_model.evaluate(test_ds)
print(f'(no LSTM) Test Loss: {test_loss}')
print(f'(no LSTM)Test Accuracy: {test_accuracy}')

# 
# mlp experiment
# 
mlp_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=False),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(1,activation=None)
])

mlp_optim = tf.optimizers.Adam()

train_vpg_mlp(mlp_model,
              optimizer=mlp_optim,
              classifier=classifier_model2,
              training_set=train_ds,
              validation_set=val_ds,
              epochs=1)
loss, acc, stat = evaluate_mlp_model(mlp=mlp_model, classifier=mlp_classifier)
means=[]
for batch in stat:
  batch_means=[]
  for pair in batch:
    batch_means.append(pair[0])
  means.append(np.mean(batch_means))
print('avg words removed: ',np.mean(means))
# print("avg removed words: ", np.mean(stat[:][0]))
print("eval acc: ", acc)

epoch=0	batch=1/625
batch=0 words deleted : 93.40625
step time for one batch: 21.2424
BERT processing time for one batch : 0.2190
Epoch 1/2
1/1 [==============================] - 0s 136ms/step - loss: 0.4998 - binary_accuracy: 0.7600 - val_loss: 0.7819 - val_binary_accuracy: 0.5714
Epoch 2/2
1/1 [==============================] - 0s 107ms/step - loss: 0.5230 - binary_accuracy: 0.7600 - val_loss: 0.7819 - val_binary_accuracy: 0.5714
batch time 28.56
epoch=0	batch=26/625
batch=25 words deleted : 142.65625
Epoch 1/2
1/1 [==============================] - 0s 132ms/step - loss: 0.5124 - binary_accuracy: 0.7200 - val_loss: 0.1934 - val_binary_accuracy: 0.8571
Epoch 2/2
1/1 [==============================] - 0s 110ms/step - loss: 0.4529 - binary_accuracy: 0.7200 - val_loss: 0.1934 - val_binary_accuracy: 0.8571
epoch=0	batch=51/625
batch=50 words deleted : 68.40625
Epoch 1/2
1/1 [==============================] - 0s 131ms/step - loss: 0.3689 - binary_accuracy: 0.8000 - val_loss: 0.4258 - val_b

NameError: ignored

LSTM Removes an average of 10 words per at the end of training
- 100+ words removed on average in first several batches

### lstm_model2 below used for smaller tests

In [16]:
import tensorflow as tf
train_ds, val_ds, test_ds = get_datasets()
VOCAB_SIZE=1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_ds.map(lambda text, label: text))

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [17]:
lstm_model2 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1,activation=None)
])

lstm_optim2 = tf.optimizers.Adam()
classifier_model2 = build_bert_model()
compile_bert_model(model=classifier_model2,
                   training_set=train_ds)


# MLP (no full training example)

In [8]:
# alternative to LSTM
# every word is an individual training example
# 
# striclty worse than LSTM
DEFAULT_STRIP_REGEX = r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']'
VOCAB_SIZE=1000
train_ds, val_ds, test_ds = get_datasets()
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_ds.map(lambda text, label: text))

mlp_model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=False),
    tf.keras.layers.Dense(64, activation='tanh'),
    tf.keras.layers.Dense(1,activation=None)
])

mlp_optim = tf.optimizers.Adam()

def mlpstep2(model, X):
  minibatch=[]
  # for idx,x in enumerate(X):
  #   string = str(x.numpy(),'utf-8').strip()
  #   string = string_ops.regex_replace(string,DEFAULT_STRIP_REGEX,'')
  #   string_input = [s for s in string.split(' ') if s]
  #   minibatch.append(model(string_input))
  #   # minibatch is split strins
  #   print(minibatch[idx])
  #   sys.exit()
  # raw_out=model(X)
  # print(raw_out[0]
  # print(raw_out.shape)

  batch_logits = model(X)
  batch_actions =[[] for _ in batch_logits]
  log_prob_sums =[0.0 for _ in batch_logits]
  _log_prob_sums =[None for _ in batch_logits]
  new_X = []
  words_removed=[]
  for idx,sentence in enumerate(batch_logits):
    temp_log_probs = []
    start=time.time()

    pol = tfp.distributions.Bernoulli(logits=sentence)
    act = pol.sample()
    batch_actions[idx] = [ac for ac in act]
    log_probs = [pol.log_prob(ac) for ac in act]
    lpkeep = pol.log_prob([0])
    lpdel = pol.log_prob([1])

    onect=0
    zeroct=0
    lp_list = []
    for i,ac in enumerate(act):
      if int(ac) == 1:
        lp_list.append(lpdel[i])
        onect+=1
      else:
        lp_list.append(lpkeep[i])
        zeroct+=1
    # print(onect,"/",zeroct)
    log_prob_sums[idx] = sum(lp_list)
    x = str(X[idx].numpy(),'utf-8')
    acts = batch_actions[idx]
    tmp_str = string_ops.regex_replace(x,DEFAULT_STRIP_REGEX,'')
    tmp_str = str(tmp_str.numpy(), 'utf-8').strip()
    tmp_list = [i for i in tmp_str.split(' ') if i]
    if (len(tmp_list) > len(acts)):
      print("ERROR: REGEX INSUFFICIENT")
      sys.exit()
    new_list = []
    addct=0
    rem=0
    for i,toke in enumerate(tmp_list):
      if acts[i] != 1:
        new_list.append(toke)
      else: rem+=1
    words_removed.append(rem)
    new_X.append(' '.join(new_list))
    if idx==0: 
      print("\tsentence action and word-removal time : ",time.time()-start)
      # print(X[0])
      # print(new_X[0])
  return batch_logits, new_X, log_prob_sums, np.mean(words_removed)

# from tensorflow.compat.v1.distributions import Multinomial as multinomial
'''
tf.random.categorical(logits=np.array([[.1,.2,.9]]),num_samples=1)
'''

import sys
import time
# 
# policy gradient (REINFORCE) training loop with LSTM model and BERT sentiment classifier
# classifier must be compiled!
# 
def train_vpg_mlp(mlp, optimizer, classifier, training_set, validation_set, epochs=5):
  total_start=time.time()
  stepct=0
  # with tf.device('/device:GPU:0'):
  for e in range(epochs):
    epoch_losses=[]
    counter=0
    trainlen = len(training_set)
    epoch_avg_del=[]
    for idx,ex_lab in enumerate(training_set):
    # for idx,ex_lab in enumerate(training_set.take(100)):
      starttime=time.time()
      example, label = ex_lab[0], ex_lab[1]
      counter+=1
      print('epoch={}\tbatch={}'.format(e,counter))
      stepct+=1
      # run through the lstm
      avg_words_del = []
      logits, new_example, logprob_sums = None, None, None
      with tf.GradientTape() as t:
        start = time.time()
        # logits, new_example, logprob_sums = step(lstm, example)
        # logits, new_example, logprob_sums = step2(lstm, example)
        logits, new_example, logprob_sums, deleted = mlpstep2(mlp, example)
        avg_words_del.append(deleted)
        print('step time ', time.time()-starttime)
        # logging for curiosity
        if e==0 and idx==0:
          print('step time for one batch: {:.4f}'.format(time.time()-start))

        # get BERT evaluations on these
        start=time.time()
        new_outs = classifier(tf.constant(new_example))
        # get bert evaluation on full sentences
        baseline_outs = classifier(example)

        if e==0 and idx==0:
          print('BERT processing time for one batch : {:.4f}'.format(time.time()-start))

        # reward LSTM with the improvement over the original sentences
        baseline_loss = tf.keras.losses.mean_squared_error(baseline_outs,tf.cast(label,float))
        loss_object = tf.keras.losses.mean_squared_error(new_outs,tf.cast(label,float))
        # loss_object = tf.keras.losses.binary_crossentropy(new_outs,tf.cast(label,float))
        adjusted_loss = loss_object-baseline_loss
        # print("loss_x' - loss_base : ",np.array(adjusted_loss))
        epoch_losses.append(adjusted_loss.numpy())

        # use BERT's loss for update:
        # use the negative losses to weight the log_probs
        weighted_logprobs = -(adjusted_loss)*logprob_sums
        # end with tf.GradientTape()

      # train bert by fitting on new_examples
      # this way, BERT and the LSTM train together
      with tf.device('/device:GPU:0'):
        history = classifier.fit(x=tf.constant(new_example),
                                  y=label,
                                  validation_split=.2,
                                  batch_size=len(new_example),
                                  epochs=2,
                                  verbose=(idx%10==0)
                                  )
      # print("BERT training accuracy : ", np.mean(history.history['binary_accuracy']))

      # get gradients and apply update
      gradients = t.gradient(weighted_logprobs, mlp.trainable_variables)
      optimizer.apply_gradients(zip(gradients,mlp.trainable_variables))

      epoch_avg_del.append(np.mean(avg_words_del))

      if idx == 0 or idx%50==0: print('batch time {:.2f}'.format(time.time()-starttime))
      # end for
    print('\n_____________________________')
    print('[{}]\t avg training loss : {}'.format(e,np.mean(epoch_losses)))
    # print('[{}]\t total train inloss : {}'.format(e,np.sum(epoch_losses)))
    with tf.device('/device:GPU:0'):
      val_loss, val_acc = classifier.evaluate(validation_set)
    print("[{}] BERT validation loss     :\t{}\n\t\tvalidation accuracy :\t{}".format(idx,val_loss, val_acc))
    print(epoch_avg_del)
  print('DONE. {:.4f}'.format(time.time()-total_start))

def remove_words_mlp(model, X):
  minibatch=[]
  batch_logits = model(X)
  batch_actions =[[] for _ in batch_logits]
  log_prob_sums =[0.0 for _ in batch_logits]
  _log_prob_sums =[None for _ in batch_logits]
  new_X = []
  removed_list=[]
  for idx,sentence in enumerate(batch_logits):
    removed=0
    temp_log_probs = []
    start=time.time()

    pol = tfp.distributions.Bernoulli(logits=sentence)
    act = pol.sample()
    batch_actions[idx] = [ac for ac in act]
    # print(onect,"/",zeroct)
    x = str(X[idx].numpy(),'utf-8')
    acts = batch_actions[idx]
    tmp_str = string_ops.regex_replace(x,DEFAULT_STRIP_REGEX,'')
    tmp_str = str(tmp_str.numpy(), 'utf-8').strip()
    tmp_list = [i for i in tmp_str.split(' ') if i]
    if (len(tmp_list) > len(acts)):
      print("ERROR: REGEX INSUFFICIENT")
      sys.exit()
    new_list = []
    addct=0
    for i,toke in enumerate(tmp_list):
      if acts[i] != 1:
        new_list.append(toke)
      else: removed+=1
    removed_list.append((removed,len(tmp_list)))
    new_X.append(' '.join(new_list))
    if idx==0: 
      print("\tsentence action and word-removal time : ",time.time()-start)

  return new_X, removed_list

def evaluate_mlp_model(mlp, classifier, testing_set):
  test_x=[]
  test_y=[]
  total=len(testing_set)
  stats = []
  for idx,example in enumerate(testing_set):
    test_y.append(example[1].numpy())
    new_x, _stats = remove_words_mlp(mlp,example[0])
    stats.append(_stats) 
    test_x.append(np.array(new_x))
    print('MLP processing batch',idx,'/',total-1)
    if idx==0:
      print("Sample MLP word removal")
      print(example[0][:5])
      print(new_x[:5])
    # print(new_x)
    # print(len(stats))
    # sys.exit()
  test_loss, test_acc = classifier.evaluate(x=test_x,y=test_y)
  return test_loss, test_acc, stats

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [9]:
mlp_classifier = build_bert_model()
compile_bert_model(model=mlp_classifier, training_set=train_ds)

In [10]:
mlp_classifier.fit(x=train_ds.shard(20,1))
train_vpg_mlp(mlp_model,
              optimizer=mlp_optim,
              classifier=mlp_classifiershard(10,1),
              training_set=train_ds,
              validation_set=val_ds,
              epochs=1)
loss, acc, stat = evaluate_mlp_model(mlp=mlp_model, classifier=mlp_classifier)
means=[]
for batch in stat:
  batch_means=[]
  for pair in batch:
    batch_means.append(pair[0])
  means.append(np.mean(batch_means))
print('avg words removed: ',np.mean(means))
# print("avg removed words: ", np.mean(stat[:][0]))
print("eval acc: ", acc)

312/312 [==============================] - 2373s 8s/step - loss: 0.5101 - binary_accuracy: 0.7340


NameError: ignored

In [113]:
loss, acc, stats = evaluate_mlp_model(mlp=mlp_model,
                                      classifier=mlp_classifier,
                                      testing_set=test_ds.shard(690,0)
)
means=[]
for batch in stats:
  batch_means=[]
  for pair in batch:
    batch_means.append(pair[0])
  means.append(np.mean(batch_means))
print('avg words removed: ',np.mean(means))

	sentence action and word-removal time :  0.07212686538696289
MLP processing batch 0 / 2
Sample MLP word removal
tf.Tensor(
[b'If one were to return to the dawn of the talking picture, one would prophesy a bright future for Harold Lloyd. Unlike his competitors, he was comedic actor trained on the legitimate stage not a performed raised in the purgatory of the music hall or vaudeville circuit. He had a good voice which matched his image. Moreover, from 1924 on, his "silent" films had incorporated sequences based on sound gags lost on the audience (e.g., the bell sequence at the Fall Frolic from THE FRESHMAN and the monkey sequence in THE KID BROTHER). Yet Lloyd\'s sound features consistently failed at the box office once the novelty of WELCOME DANGER had ebbed. Lloyd blamed his fall on many external sources, but never realized that the Glass character\'s enemy was not sound but the Great Depression. Pre-Depression audiences, giddy with optimism, may have rooted for this ambitious go-get